## A) Download der Wikipedia Daten

Das herunterladen der Dump-Datei kann über einen der folgenden Befehle ausgeführt werden:

```sh
# Please run the commands from the root directory of this repository. To check run
$ pwd
/path/to/repositories/forschungsseminar-nlp
# Download subset for testing/ development
$ sh ./scripts/download-dump.sh
# Dowload everything for application
$ sh ./scripts/download-large-dump.sh
```

>Diese Befehle **müssen** im Root-Directory dieses Repositories ausgeführt werden!

Zur Auswahl stehen hier

1. eine Dumpdatei mit einer kleineren Größe von ca. 5GB (im Prozess entstehend) für ein effizientes Arbeiten und
2. die allumfassende Dumpdatei mit einer Größe von ca. 28GB (im Prozess entstehend)

Über die Variable `ONLY_SAMPLE` kann bestimmt werden ob nur ein Subset der Daten beispielhaft verarbeitet werden oder der komplette große Dump verarbeitet wird.

In [1]:
ONLY_SAMPLE = False

### Extraktion des Dumps:

Die Bibliothek "MediaWiki XML Processing" oder auch "mwxml" ermöglicht ein vereinfachtes Streamen XML-Dumps über eine Abstrahierung.
Über den mwxml.Dump werden die Wikipediaartikel auf eine iterative Weise über mwxml.Page‘s zugänglich gemacht.
Im Folgenden werden über die angegebene Datei (entweder die kleine oder große XML-Datei) ein mwxml-Dump erstellt, welcher eben diese Attribute bereitstellt.
Um die Funktionalität und Fehlerfreiheit zu prüfen werden im Folgenden zudem die Info der Seite und der Name der untersuchten Datenbank ausgegeben. Zu erwarten wären hier Hinweise auf die deutsche Wikipediafassung.

In [2]:
# Importieren der Bibliotheken und Module
import mwxml # Für Untersuchung der XMLs
import re # Für spätere regex-Operationen
from tqdm import tqdm # Für die Integration von Fortschrittsbannern, Ladebalken etc.

def loadDump():
    if ONLY_SAMPLE:
        dump = mwxml.Dump.from_file(open("../data/dewiki-20220520-pages-articles-multistream1.xml"))
    else:
        #dewiki-20220620-pages-articles-multistream.xml als Alternative, falls großer Dump genutzt werden soll.
        dump = mwxml.Dump.from_file(open("../data/dewiki-20220620-pages-articles-multistream.xml"))
    return dump

dump = loadDump()
dump_len = 155401 if ONLY_SAMPLE else 5235974 # Da uns durch ausprobieren die Längen beider Dateien bekannt sind, kann hier Zeit gespart werden indem die Längen hardcoded werden. Muss für neue Dateien entsprechend entfert werden
#dump_len = sum(1 for _ in dump) # Das verbraucht den 'items' generator des dumps, weshalb 'dump' neu geladen 
#dump = loadDump()
dump.site_info.name, dump.site_info.dbname, dump_len # Ausgabe der Informationen zum Dump

('Wikipedia', 'dewiki', 5235974)

## B) Sammeln der Firmennamen aus Dump

Im Folgenden wird vorläufig eine später verwendete Methodik und Funktion zur weiteren Verbesserung der Firmenklassifikation implementiert.
Für eine spätetere Ausräumung von Unsicherheiten in deutscher Wiki werden sowohl deutsches NER-Modell auf deutscher Wiki und englisches Modell auf englischer Wiki genutzt. 
Auch mit deutschem Modell auf rein deutscher Wikipedia möglich.

In [3]:
!python -m spacy download en_core_web_sm # Herunterladen des englischen Spacy-Sprachmodells per Bash-Skript - erfahrungsgemäß bessere Zuverlässigkeit.
!python -m spacy download de_core_news_sm # Herunterladen des deutschsprachigen Spacy-Sprachmodells per Bash-Skript

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 36.5 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/home/tobias/.cache/pypoetry/virtualenvs/forschungsseminar-nlp-PHREsQ0p-py3.8/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 26.2 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/home/tobias/.cache/pypoetry/virtualenvs/forschungsseminar-nlp-PHREsQ0p-py3.8/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


Nachdem die nötigen Modelle heruntergeladen wurden, kann nun die Funktion zur Verbesserung der Genauigkeit implementiert werden.

In [4]:
# Firmenerkennung über NER
import pandas as pd
import spacy # spacy ist eine Bibliothek für fortgeschrittene Natürliche Sprachverarbeitung
import wikipediaapi # Vorteil: sowohl für 'de' als auch 'en' nutzbar: die wikipediapi ist eine Implementierung der Wikipedia-Api als eine "Wikipedia Python API". So können sehr effizient Zusammenfassungen und Seiten zu gesuchten Titeln angefordert werden.

en_nlp = spacy.load("en_core_web_sm") # Laden des englischen Spacy-Modells für NER
de_nlp = spacy.load("de_core_news_sm") # Laden des deutschen Spacy-Modells für NER
en_wiki = wikipediaapi.Wikipedia('en') # Laden der englischen wikipediaapi-Version
de_wiki = wikipediaapi.Wikipedia('de') # Laden des deutschen wikipediaapi-Version

# Regex-Pattern
pattern = re.compile('.*\[\[Kategorie:(.*)\]\].*') # Regex-Kriterium für Erkennung von "Kategorie" Feld, welches sich unten auf jeder Wikipedia-Seite befindet.
list_of_words = ['hersteller', 'unternehmen', 'Unternehmen', 'Hersteller'] # Sollten keywords dieser Liste in dem "Kategorien"-Feld enthalten sein, liefert dies Hinweise darauf, dass es sich bei dem Thema des Artikels um ein Unternehmen handelt.
uncertainty_markers = ['unternehmens', 'Unternehmens'] # Allerding gibt es auch Ausschlusskriterien, die zunächst nur auf mögliche Unsicherheiten hinweisen: könnte es sich in der Kategorie z.B. um einen Untenehmensberater handeln und nicht direkt ein Unternehmen?
words_re = re.compile("|".join(list_of_words)) # Erstellung des Regex-Musters mit Liste
uncertainty_re = re.compile("|".join(uncertainty_markers)) # Erstellung des Regex-Musters mit Liste


def reduceUncertainty(page_title):
    """Diese Funktion soll abgerufen werden können, falls bei einer erkannten Entität nicht klar ist, ob es sich um eine Firma handelt. Hierdurch sollen Unsicherheiten reduziert werden.
    """
    en_page = en_wiki.page(page_title) # Das Funktionsattribut page_title enthält den gesuchten Firmenname/Wikipediatitel und wird hier in der englischen API gesucht. 
    de_page = de_wiki.page(page_title) # Das Funktionsattribut page_title enthält den gesuchten Firmenname/Wikipediatitel und wird hier über die deutsche API gesucht.

    if not en_page.exists() and not de_page.exists(): # Englische und Deutsche Wikipedia-Seite existiert nicht
        return False # gibt False("company not exists") zurück, wenn die Seite nicht gefunden werden kann. So werden nur Seiten entfernt über die wir Zusatzwissen erlangen konnten und am wenigsten drastisch eingegriffen. Je nach Business Case kann dies anders Implementiert werden.

    # Deutsche Seite existiert aber Englische Seite existiert nicht - in diesem Fall muss idealerweise ein deutsches NER-Modell verwendet werden
    doc =  en_nlp(en_page.summary) if en_page.exists() else de_nlp(de_page.summary) # Die Zusammenfassung sollte in den meisten Fällen für die Urteilsfällung ausreichen
    org_list = [] # Diese Liste wird leer erklärt, um später alle erwähnten Organisationen in dem Betrefflichen Firmenartikel zu erkennen.
    for ent in doc.ents: # Alle erkannten Entitäten werden hier gesammelt
        if (ent.label_ == 'ORG') and (page_title in ent.text): # ... Und anschließend gefiltert: Zunächst sollen nur erkannte Organisationen behalten werden, welche zudem in ihrem Eigennamen den Titel der untersuchten Seite als String enthalten. Zukünftig könnten hier erweiterte Regex-Operationen verwendet werden.
            org_list.append(ent) # Nur diese, den Kriterien unterliegende Entitäten, werden der Liste von erkannten Organisationen hinzugefügt.
    return len(org_list) > 0

def getPageText(page: mwxml.iteration.page.Page) -> list:
    """
    Diese Funktion extrahiert die Texte aller Revisions einer Wikipedia Page des Dumps. Dies ist notwendig, da mwxml ausschließlich mit Iteratoren arbeitet, welche lediglich einmal benutzt werden können. So werden die Texte extrahiert und sie sind wiederholbar iterabel.
    """
    texts = [revision.text for revision in page]
    return texts

def getCategoriesFromPage(texts: list) -> list:
    """
    Diese Funktion extrahiert Wikipedias Kategorien aus den Texten 
    """
    # Extrahieren von Kategorien, dazu wird zuerst über alle Texte der Revisions der Wikipedia Page iteriert und darauf über alle matches des ersten Regex-Patterns, welches auf Kategorienfeld hindeutet oder nicht.
    categories = [match.group(1) for text in texts for match in pattern.finditer(text)]
    return categories

def isCompany(categories: list, page_title: str) -> bool:
    """
    Diese Funktion zeigt ob die Kategorien auf ein Unternehmen hindeuten oder nicht
    """
    
    line = ",".join(categories) # Um diese Kategorien weiter zu verarbeiten werden sie als ein großer String zusammen gefasst

    if not words_re.search(str(line)): # Anwendung des zweiten Regex-Patterns, um grob zu erkennen, ob es sich bei der seite um die eines Unternehmens handeln könnte, oder nicht.
        return False # Page überspringen, wenn es sich definitiv nicht um ein Unternehmen handelt
    if uncertainty_re.search(str(line)): # Das nächste Regex-Pattern weist auf Unsicherheiten hin, da oft "Unternehmens" + "andere Wortteil", darauf hindeuten, dass Artikel selbst nur etwas, mit der Firma im Zusammenhang setehendes, thematisiert.
        return reduceUncertainty(page_title) # Um hier mehr Gewissheit zu erlangen werden in den vereinzelten Fällen die eigenen Wikipediaartikel der Entitätenuntersucht. True wird zurückgegeben, wenn es sich um Firma handeln könnte.
    # Wenn vorher nicht ausgefiltert wurde handelt es sich um ein Unternehmen
    return True

Das Obiges Skript durchläuft also den folgenden logischen Prozess bzw. Algorithums:

**Existiert Artikel im Englischen Wiki?**
- **JA:** versuche Organisationen mit englischem Modell zu erkennen
- **NEIN:** existiert Artikel im Deutschen Wiki? 
    - **JA:** versuche Organisationen mit deutschem Modell zu erkennen
    - **NEIN:** aus Toleranz als Firma oder nicht Firma zählen.

Im Folgenden wird diese Methode kurz getestet, um Sie später in der Sammlung von Firmennamen anwenden zu können:

In [5]:
# Tests:

page = en_wiki.page("Aldi") # Benennung der gesuchten Wikipediaseite als "Aldi"

print(page.exists()) # Überprüfung ob Seite existiert

print("Page - Summary: %s" % page.summary[0:60]) # Ausgabe eines Zusammenfassungteils

doc = en_nlp(page.summary) # Laden der Seitenzusammenfassung in englisches NLP-Modell

for ent in doc.ents: # Iterieren über erkannte Entitäten
    print(ent.text, ent.label_) # Ausgabe der Entitätenart und des Titels

# Nun kann sich ein Bild gemacht werden, ob die Zusammenfassung Entitäten liefert, die darauf hindeuten, dass es sich bei "Aldi" um eine Firma handelt.

True
Page - Summary: Aldi (stylised as ALDI) is the common company brand name of 
ALDI ORG
two CARDINAL
German NORP
10,000 CARDINAL
20 CARDINAL
Karl PERSON
Theo Albrecht PERSON
1946 DATE
Essen GPE
two CARDINAL
1960 DATE
Aldi Nord PERSON
Essen GPE
Aldi Süd PERSON
Mülheim GPE
1962 DATE
Aldi PERSON
Albrecht Diskont ORG
Germany GPE
Aldi Nord PERSON
Aldi Süd PERSON
1966 DATE
Aldi Nord PERSON
Aldi Einkauf GmbH & Co. ORG
Aldi Sud PERSON
German NORP
Aldi Nord's PERSON
35 CARDINAL
about 2,500 CARDINAL
Germany GPE
Aldi Süd's PERSON
32 CARDINAL
1,900 CARDINAL
Germany GPE
Aldi Nord PERSON
Denmark GPE
France GPE
Benelux GPE
Portugal GPE
Spain GPE
Poland GPE
Aldi Süd PERSON
Ireland GPE
the United Kingdom GPE
Hungary GPE
Switzerland GPE
Australia GPE
China GPE
Italy GPE
Austria GPE
Slovenia GPE
Aldi Nord PERSON
Aldi Süd PERSON
Aldi PERSON
the United States GPE
1,600 CARDINAL
2017 DATE
U.S. GPE
Aldi NORP
Germany GPE
2020 DATE
Aldi Nord PERSON
Aldi Süd PERSON
2018 DATE
two CARDINAL
2022 DATE


Nun können mithilfe der obigen Funktionen und mithilfe von im Folgenden erklärten regex-Mustern mit erhöhter Zuverlässigkeit Firmen aus dem Dump extrahiert werden.
Diese werden anschließend der Textdatei `title-categories-map.txt` abgelegt.

In [6]:
companies = [] # Liste welche später genutzt wird um einen DataFrame zu erstellen. Speichert Titel und Kategorien in einem Dict je Unternehmen
for i, page in enumerate(tqdm(dump, total=dump_len, desc="Finding companies...")): # Iteration über Seiten im Dump
    try: # Try-Catch-Block, damit einzelne Fehler nicht zu Abbruch des Schreibprozesses führen - gerade bei großem Dump sehr wichtig
        texts = getPageText(page)
        categories = getCategoriesFromPage(texts)
        is_company = isCompany(categories, page.title)

        # Wenn Page kein Unternehmen ist soll sie übersprungen werden und nicht zur Liste der Unternehmen hinzugefügt werden
        if not is_company:
            continue

        # Einfügen in die Liste der Unternehmen
        companies.append({
            "title": page.title,
            "categories": ", ".join(categories),
            "dump_index": i
        })

        # Abbrechen wenn nur ein einziges Sample geladen werden soll
        if ONLY_SAMPLE:
            break
    except KeyboardInterrupt as e:
        raise e
    except SystemExit as e:
        raise e
    except Exception as e:
        tqdm.write(f"Fehler bei {page.title} ({i})")
        tqdm.write(repr(e))
        continue

companies = pd.DataFrame(companies)
companies.to_feather("../data/company-categories.feather")
companies

Finding companies...: 100%|██████████| 5235974/5235974 [2:11:34<00:00, 663.25it/s] 


,title,categories,dump_index
0,Apple,"Apple| , Hardwarehersteller (Vereinigte Staate...",121
1,Aldi,"Aldi| , Abkürzung, Einzelhandelsunternehmen (D...",180
2,E-Plus,"Ehemaliger Mobilfunkanbieter, Telekommunikatio...",960
3,First National,Ehemalige Filmgesellschaft (Vereinigte Staaten...,1004
4,GfK (Unternehmen),"Marktforschungsunternehmen, Dienstleistungsunt...",1243
...,...,...,...
71225,Aspen Holdings Logo.svg,Datei:Logo (Unternehmen aus Südafrika),5235225
71226,Schürfeld,"Papierhersteller, Unternehmen (Hamburg)",5235253
71227,GUSCO,Handelsunternehmen (Hamburg),5235255
71228,Pulverfabrik Skodawerke-Wetzler,"Österreichische Wirtschaftsgeschichte, Ehemali...",5235474


In [7]:
company_dump_idx = set(companies["dump_index"]) # Ist später nützlich, um die recht aufwendige Funktionen 'getCategoriesFromPage' und 'isCompany' nicht nochmals auszuführen, sondern beim erneutem iterieren über den dump lediglich `if not i in company_dump_idx: continue` aufgerufen werden kann. Speicherung als set, da sets deutlich schneller als pandas Series und normale Listen sind wenn es um solche aufrufe geht.

In [8]:
del companies # Speicherplatz freigeben

## C) Extraktion der Infobox

Der Aufbau einer Infobox eines Unternehmens in dessen Wikipedia Artikel ist streng definiert. Die Infobox beginnt mit zwei geschweiften Klammern und dem Typ des Eintrags, in diesem Fall `{{{Infobox Unternehmen`. Sie enthält maximal 14 Einträge: Name, Logo, Unternehmensform, ISIN, Gründungsdatum, Auflösungsdatum, Sitz, Leitung, Mitarbeiterzahl, Umsatz, Stand, Branche, Website. Erforderlich sind hiervon Unternehmensform sowie Sitz. 

Die Infobox wird mit folgendem Regex Pattern definiert: `((?<={{Infobox Unternehmen.).*?(?<=Homepage).*?(}})).`
Es wird der Inhalt nach {{Infobox Unternehmen bis einschließlich dem letzten Eintrag Homepage.*?}} ausgelesen. 

> Verbesserungsvorschlag: Einträge einzeln auslesen, sodass nur die Parameter ausgegeben werden, die einen Wert besitzen.

In [9]:
dump = loadDump() # Da im Aufgabenteil B der dump Iterator aufgebraucht wurde, muss dieser neu geladen werden
infobox = re.compile('((?<={{Infobox Unternehmen.).*?(?<=Homepage).*?(}}))') # Regex Pattern erkennt den Text der Infobox zwischen den Keywörtern Unternehmen und Homepage

def isCompanyByIdx(i, page, texts):
    """
    Funktion welche deutlich schneller entscheidet ob eine Page übersprungen werden muss. Nutz den in Aufgabe B erstellen Dump-Index. Wenn ONLY_SAMPLE == True ist wurde dieser Index nicht vollständig erstellt, weshalb auf die normale Methode zurückgegriffen wird.
    """
    if ONLY_SAMPLE:
        categories = getCategoriesFromPage(texts)
        is_company = isCompany(categories, page.title)
        return is_company

    return i in company_dump_idx


with open("../data/infobox.txt", mode="w") as outfile:
    for i, page in enumerate(tqdm(dump, total=dump_len, desc="Extracting infoboxes...")):
        try: # Try-Catch-Block, damit einzelne Fehler nicht zu Abbruch des Schreibprozesses führen - gerade bei großem Dump sehr wichtig
            texts = getPageText(page)

            # Wenn Page kein Unternehmen ist soll sie übersprungen werden
            if not isCompanyByIdx(i, page, texts):
                continue

            line = ''
            for text in texts:
                text = re.sub(r'\n', '', text) # Linebreaks entfernen
                for match in infobox.finditer(text):
                    line += match.group(1) + "\n" 
                    line = " ".join(line.split()) # Löschen mehrerer Whitespaces
                    outfile.write(line+"\n")

            # Abbrechen wenn nur ein einziges Sample geladen werden soll
            if ONLY_SAMPLE:
                break

        except KeyboardInterrupt as e:
            raise e
        except SystemExit as e:
            raise e
        except Exception as e:
            tqdm.write(f"Fehler bei {page.title} ({i})")
            tqdm.write(repr(e))
            continue

Extracting infoboxes...: 100%|██████████| 5235974/5235974 [12:54<00:00, 6763.85it/s] 


In [10]:
num_lines = sum(1 for line in open('../data/infobox.txt'))
num_lines
#Output: 24771

24771

### Unternehmen ohne Infobox

Nun soll auch für Unternehmen ohne Infobox deren Namen sowie dessen Webseite ausgegeben werden. 

Hierfür wird im ersten Schritt geprüft, ob der Wikipedia Artikel des Unternehmens das Wort Infobox enthält. Dies wird mit folgendem Regex Pattern geprüft: `(?!.*?Infobox)^.*$` 
Enthält der Artikel das Wort nicht, so erkennt das Regex Pattern den gesamten Artikel. 
Wird der Artikel als Unternehmensartikel klassifiziert und enthält er das Wort Infobox nicht, wird der Artikelname (=Unternehmensname) ausgegeben. 

Um nun auch die Webseite des Unternehmens ausgeben zu lassen, wird folgendes Regex Pattern angewandt: 
`.*((?<=Weblinks...).*?(?=...Einzelnachweise)).*`
Dies gibt alle in der Sektion Weblinks eingetragnenen Webseiten aus, worunter oftmals auch die offizielle Unternehmenswebseite fällt. 

> Verbesserungsvorschlag: Es werden viele Artikel ausgelesen, die die Kategorie Unternehmsart enthalten, aber keine Unternehmen sind. Diese sind oftmals allgemeine Informationsseiten und enthalten daher auch keine Infobox, weshalb sie vermehrt ausgelesen werden. Darüber hinaus wird für die Unternehmenswebseite die ganze Sektion der Weblinks ausgelesen, die oftmals auch andere weiterführende Links enthält. Eine Verbesserungsmöglichkeit bestände darin, den Namen des Unternehmens in den Links zu suchen und nur die Links, die den Namen enthalten, auszugegeben. 

In [11]:
#Companies without infobox
dump = loadDump() # Da im Aufgabenteil B der dump Iterator aufgebraucht wurde, muss dieser neu geladen werden

pattern_website = re.compile('.*((?<=Weblinks...).*?(?=...Einzelnachweise)).*') 

with open("../data/company_name&website.txt", mode="w") as outfile:
    for i, page in enumerate(tqdm(dump, total=dump_len, desc="Extracting companies without infoboxes...")):
        try: # Try-Catch-Block, damit einzelne Fehler nicht zu Abbruch des Schreibprozesses führen - gerade bei großem Dump sehr wichtig
            texts = getPageText(page)

            # Wenn Page kein Unternehmen ist soll sie übersprungen werden
            if not isCompanyByIdx(i, page, texts):
                #pass
                continue
            

            found = False
            for text in texts:
                text = re.sub(r'\n', '', text) # Linebreaks entfernen

                # Unternehmen mit Infoboxen überspringen
                if infobox.search(text):
                    continue

                for match in pattern_website.finditer(text): # Suche nach Weblinks
                    outfile.write(page.title+"\n") # Ausgabe des Page Titels
                    website = match.group(1)
                    outfile.write(website+"\n"+"\n")
                    found = True

            # Abbrechen wenn nur ein einziges Sample geladen werden soll
            if ONLY_SAMPLE and found:
                break
        
        except KeyboardInterrupt as e:
            raise e
        except SystemExit as e:
            raise e
        except Exception as e:
            tqdm.write(f"Fehler bei {page.title} ({i})")
            tqdm.write(repr(e))
            continue

Extracting companies without infoboxes...: 100%|██████████| 5235974/5235974 [1:33:44<00:00, 930.87it/s]  


In [12]:
companies_without_infobox = sum((1//3) for line in open('../data/company_name&website.txt'))
companies_without_infobox
#Output: 15837

15837.000000010925

Um nun eine Kurzbeschreibung des Unternehmens zu erhalten, wird der erste Satz des Artikels erfasst. Dieser enthält bei Unternehmen den Unternehmensnamen eingefasst in ''' '''. Aus diesem Grund wird folgendes Regex Pattern angewandt, um den ersten Satz auszulesen: 
((?<=\''' ).*?(?<=\.)+) 

In [13]:
dump = loadDump() # Da im Aufgabenteil B der dump Iterator aufgebraucht wurde, muss dieser neu geladen werden
pattern_firstsentence = re.compile('((?<=\''' ).*?(?<=\.))')
#Erster Satz fängt meistens mit dem Firmennamen in ''' ''' an. Dann wird der erste Punkt gesucht.

with open("../data/firstsentence.txt", mode="w") as outfile:
    for i, page in enumerate(tqdm(dump, total=dump_len, desc="Extracting short description...")):
        try: # Try-Catch-Block, damit einzelne Fehler nicht zu Abbruch des Schreibprozesses führen - gerade bei großem Dump sehr wichtig
            texts = getPageText(page)

            # Wenn Page kein Unternehmen ist soll sie übersprungen werden
            if not isCompanyByIdx(i, page, texts):
                continue

            firstsentence = ""
            for text in texts:
                text = re.sub(r'\n', '', text) # Linebreaks entfernen

                firstsentence = ""
                for match in pattern_firstsentence.finditer(text):
                    firstsentence += match.group(1)
                    firstsentence.replace(r'\*', '')
                outfile.write(page.title+"\n")
                outfile.write(firstsentence+"\n"+"\n")

            # Abbrechen wenn nur ein einziges Sample geladen werden soll
            if ONLY_SAMPLE:
                break
            
        except KeyboardInterrupt as e:
            raise e
        except SystemExit as e:
            raise e
        except Exception as e:
            tqdm.write(f"Fehler bei {page.title} ({i})")
            tqdm.write(repr(e))
            continue

Extracting short description...: 100%|██████████| 5235974/5235974 [11:51<00:00, 7357.68it/s] 


In [14]:
firstsentences = sum((1//3) for line in open('../data/firstsentence.txt'))
firstsentences
#Output: 71230

71230.00000010787